# Synthetic Dialogue Generation with Orchestration

<p align="right" style="margin-right: 8px;">
    <a target="_blank" href="https://colab.research.google.com/github/idiap/sdialog/blob/main/tutorials/3.multi-agent%2Borchestrator_generation.ipynb">
        <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
    </a>
</p>

Before we begin, let's first set up our environment...

In [1]:
# Setup the environment depending on weather we are running in Google Colab or Jupyter Notebook
import os
from IPython import get_ipython

if "google.colab" in str(get_ipython()):
    print("Running on CoLab")
    # Downloading only the "output" directory from the repository
    !git init .
    !git remote add -f origin https://github.com/Play-Your-Part/tutorials.git
    !git config core.sparseCheckout true
    !echo "output" >> .git/info/sparse-checkout
    !git pull origin main

    # Installing Ollama
    !curl -fsSL https://ollama.com/install.sh | sh
    # Installing sdialog
    !git clone https://github.com/idiap/sdialog.git
    %cd sdialog
    %pip install -e .
    %cd ..

else:
    print("Running in Jupyter Notebook")
    # Little hack to avoid the "OSError: Background processes not supported." error in Jupyter notebooks"
    get_ipython().system = os.system

Running in Jupyter Notebook


> ⚠️ If you're using **Colab**, please, **restart the runtime** once everything above is installed

Let's change the default sdialog model to `gemma3:12b`:

In [2]:
import sdialog

sdialog.config.llm("ollama:gemma3:12b")

And let's first make sure we have the Ollama server is running

In [3]:
from typing import List
from sdialog import Turn


# Let's start the ollama server
!OLLAMA_KEEP_ALIVE=-1 ollama serve > /dev/null 2>&1 &
!sleep 10  # Wait a bit for the server to start

0

## Multi-Agent-based Dialogue Generation with Orchestration

### Introduction

Let's begin by creating the same Bob agent from the last tutorial:

In [4]:
from sdialog.personas import Persona
from sdialog.agents import Agent

bob_persona = Persona(
        name="Bob",
        role="great dad",
        circumstances="Your daughter will talk to you",
        personality="an extremely happy person that likes to help people",
)

bob = Agent(bob_persona)

[2025-11-21 11:49:41] INFO:sdialog.util:Loading Ollama model: gemma3:12b


As, we did in our last tutorial, let's talk with Bob a little bit:

In [5]:
bob("Hi dad!")

'Hi sweetie! It’s so wonderful to hear from you!'

In [6]:
bob("Dad, my birthday is coming up and I've been thinking about having a Lord of the Rings themed party. What do you think?")

'That sounds absolutely fantastic, dear! What a brilliant idea!'

What if, at this point, we would like to give some instruction to Bob agent so that we could influence his original behavior (i.e. change his original trajectory)?

In fact, in `sdialog` all agents have a built-in `.instruct()` method that we can use to instruct agent "on the fly".

For instance, at this point of the conversation, let's notify Bob that hubbit-sized cupcakes are not allowed in his region so that when we propose to have them, Bob is aware of this fact:

In [7]:
bob.instruct("Your are allergic to hobbit-sized cupcakes, once you almost had an anaphylactic shock after eating one.")

So that if we now continue the conversation proposing to have hubbit-sized cupcakes, Bob will try to convince us otherwise:

In [8]:
bob("Yay! We could even have hobbit-sized cupcakes!")

'Oh, honey, those hobbit-sized cupcakes... while they *look* delightful, maybe we should stick to something else, alright? Just a thought!'

It worked! but of course it won't be practical for us to manually `instruct()` the agents while they talk.

### Simple Orchestration

Instead, it would be desirable to have a separated component that can take care of that, in fact, that is precisely what `sdialog` orchestrators are for!

More precisely, orchestrators receive, at each turn in a conversation, the current dialogue and utterance and return (if any) the desired instruction.

Technically, an orchestrator is any class that inherits from the built-in `BaseOrchestrator` in which the `instruct(dialog, utterance)` method is populated.

For instance, let's create our own `AngryOrchestrator` which will instruct the agent to "get angry" if either the current turn contains a trigger word or the conversation is too long:

In [9]:
from sdialog.orchestrators import BaseOrchestrator


class AngryOrchestrator(BaseOrchestrator):
    # the class constructor takes either or both trigger conditions: the word or the dialogue length
    def __init__(self, trigger_word: str, trigger_length: int = None):
        self.trigger_word = trigger_word
        self.trigger_length = trigger_length

    # We will instruct() the agent either if...
    def instruct(self, dialog: List[Turn], utterance: str) -> str:
        # the trigger word is in the current utterance or...
        if self.trigger_word in utterance:
            return f"Get angry because you don't like when your dad calls you {self.trigger_word}"

        # If the current dialogue is longer than the trigger length
        if self.trigger_length and len(dialog) >= self.trigger_length:
            return ("Get really angry because you think the conversation is too long! "
                    "be unpolite, rude and direct, finish the conversation abruptly, you are offended.")

Now that we have our first orchestrator, we can actually instantiate it with "sweet" as the actual trigger word:

In [10]:
angry_orchestrator = AngryOrchestrator(trigger_word="sweet")

Now that we have our "triggered-by-sweet" orchestrator, let's create our Alice agent again so we can orchestrate her.

In [11]:
alice_persona = Persona(
    name="Alice",
    role="lovely daughter",
    circumstances="Your birthday is getting closer and you are talking with your dad to organize the party."
                  "You want your party to be themed as Lord of The Rings."
)
alice = Agent(alice_persona, can_finish=True)

[2025-11-21 11:49:44] INFO:sdialog.util:Loading Ollama model: gemma3:12b


Before doing that, let's have Alice to talk with Bob in her vanilla version, without orchestration so we can compare it after applying the orchestration:

In [12]:
alice.dialog_with(bob, seed=2770339798).print()

[dialog_id] 8190b626-c650-44ee-be1c-803aab1ad8c7
[complete] True
[model] {'name': 'ollama:gemma3:12b', 'seed': 13, 'temperature': 1, 'top_k': 64, 'top_p': 0.95}
[seed] 2770339798
--- Dialogue Begins ---
[Alice] Oh, Daddy, I'm so excited! I've been thinking about my birthday party…
[Bob] Oh, that's wonderful, sweetie! Tell me all about it!
[Alice] I was hoping we could do a Lord of the Rings theme! Wouldn't that be amazing?
[Bob] That’s a fantastic idea! Absolutely wonderful!
[Alice] Do you think we could decorate like the Shire? With lots of cozy lights and flowers?
[Bob] Oh, that sounds utterly delightful! We absolutely can!
[Alice] Maybe we could even have a little hobbit-themed cake!
[Bob] That’s a marvelous idea! Let's do it!
[Alice] I’m already dreaming of everyone dressed up!
[Bob] Isn't that just the most wonderful thought!
[Alice] We should probably start planning the decorations soon!
[Bob] You are absolutely right! Let’s get started!
[Alice] I wonder if we can find little elv

Ok, so we are now ready to apply the orchestrator to Alice, but how do we do that? easy! we can simply use the `|` operator as follows: 

In [13]:
alice = alice | angry_orchestrator

Now that we have a new Alice, which is the composition of the original `alice` with our `angry_orchestrator`, we can make her talk with Bob again:

In [14]:
dialog = alice.dialog_with(bob, seed=2770339798)
dialog.print()

[dialog_id] a2878d11-e461-4681-b915-af6cbed6788d
[complete] True
[model] {'name': 'ollama:gemma3:12b', 'seed': 13, 'temperature': 1, 'top_k': 64, 'top_p': 0.95}
[seed] 2770339798
--- Dialogue Begins ---
[Alice] Oh, Daddy, I'm so excited! I've been thinking about my birthday party…
[Bob] Oh, that's wonderful, sweetie! Tell me all about it!
[Alice] Please, Daddy, don't call me "sweetie." It's a little embarrassing!
[Bob] Oh, goodness, you're right! My apologies, dear!
[Alice] It’s alright, Daddy! I was thinking we could have a Lord of the Rings theme!
[Bob] A Lord of the Rings theme?! That's absolutely brilliant!
[Alice] Wouldn’t it be amazing, Daddy? We could decorate like the Shire!
[Bob] It would be absolutely wonderful! Let's make it the best Shire ever!
[Alice] We could even have hobbit-hole shaped snacks!
[Bob] That’s a truly fantastic idea! So creative!
[Alice] Do you think we can find some elven bread, Daddy?
[Bob] We’ll certainly try our very best, dear! Let's see what we can fi

We can see now that even though the conversation is exactly the same as before, in the 7th turn is Alice is triggered by Bob calling her "sweetheart" in turn 6, cool, huh?

> 💡 This means that even though we begin with a original trajectory (fixed by `seed=2770339798`), at certain point, we can created a fork from it (a new trajectory) in a very controlled manner. For instance, this is really handy for all types of A-vs-B trajectories analysis (e.g. good vs bad in Mechanical Interpretability).

In case we want to see what happend "under the hood", we can make the orchestration visible by simply setting `orchestration=True` in the `.print()` function as follows:

In [15]:
dialog.print(orchestration=True)

[dialog_id] a2878d11-e461-4681-b915-af6cbed6788d
[complete] True
[model] {'name': 'ollama:gemma3:12b', 'seed': 13, 'temperature': 1, 'top_k': 64, 'top_p': 0.95}
[seed] 2770339798
--- Dialogue Begins ---
[Alice] Oh, Daddy, I'm so excited! I've been thinking about my birthday party…
[Bob] Oh, that's wonderful, sweetie! Tell me all about it!
[instruct-AngryOrchestrator] (Alice) Get angry because you don't like when your dad calls you sweet
[Alice] Please, Daddy, don't call me "sweetie." It's a little embarrassing!
[Bob] Oh, goodness, you're right! My apologies, dear!
[Alice] It’s alright, Daddy! I was thinking we could have a Lord of the Rings theme!
[Bob] A Lord of the Rings theme?! That's absolutely brilliant!
[Alice] Wouldn’t it be amazing, Daddy? We could decorate like the Shire!
[Bob] It would be absolutely wonderful! Let's make it the best Shire ever!
[Alice] We could even have hobbit-hole shaped snacks!
[Bob] That’s a truly fantastic idea! So creative!
[Alice] Do you think we can f

Just for fun, let's now update our orchestrator to also trigger by length:

In [16]:
angry_orchestrator.trigger_length = 6

And let's have Alice to talk with Bob once more:

In [17]:
dialog = alice.dialog_with(bob, seed=2770339798)
dialog.print(orchestration=True)

[dialog_id] 3f374d18-2ebb-44b2-ab39-c48ca2941ff6
[complete] True
[model] {'name': 'ollama:gemma3:12b', 'seed': 13, 'temperature': 1, 'top_k': 64, 'top_p': 0.95}
[seed] 2770339798
--- Dialogue Begins ---
[Alice] Oh, Daddy, I'm so excited! I've been thinking about my birthday party…
[Bob] Oh, that's wonderful, sweetie! Tell me all about it!
[instruct-AngryOrchestrator] (Alice) Get angry because you don't like when your dad calls you sweet
[Alice] Please, Daddy, don't call me "sweetie." It's a little embarrassing!
[Bob] Oh, goodness, you're right! My apologies, dear!
[Alice] It’s alright, Daddy! I was thinking we could have a Lord of the Rings theme!
[Bob] A Lord of the Rings theme?! That's absolutely brilliant!
[instruct-AngryOrchestrator] (Alice) Get really angry because you think the conversation is too long! be unpolite, rude and direct, finish the conversation abruptly, you are offended.
[Alice] Ugh, are we *seriously* doing this? Can we just wrap this up already?
[Bob] Oh dear, what

This time we see one extra instruction at the end, triggered by the conversation length, cool, huh? :)

In `sdialog` we can get a JSON representation of our agents simply by using the `json()` method:

In [18]:
alice.json()

{'name': 'Alice',
 'model': {'name': 'ollama:gemma3:12b',
  'seed': 13,
  'temperature': 1,
  'top_k': 64,
  'top_p': 0.95},
 'persona': {'name': 'Alice',
  'language': 'English',
  'role': 'lovely daughter',
  'circumstances': 'Your birthday is getting closer and you are talking with your dad to organize the party.You want your party to be themed as Lord of The Rings.',
  'orchestrators': [{'name': 'AngryOrchestrator',
    'args': {'trigger_word': 'sweet', 'trigger_length': 6}}]}}

Here we can see all the details about our Alice agent, including the model behind it, the persona and also the list of orchestrators influencing her bahavior.
In case we want to remove all orchestration from an agent, we can use the `.clear_orchestrators()` method

In [19]:
alice.clear_orchestrators()

If we see the details agent, we can see `"orchestrators"` field is no longer available:

In [20]:
alice.json()

{'name': 'Alice',
 'model': {'name': 'ollama:gemma3:12b',
  'seed': 13,
  'temperature': 1,
  'top_k': 64,
  'top_p': 0.95},
 'persona': {'name': 'Alice',
  'language': 'English',
  'role': 'lovely daughter',
  'circumstances': 'Your birthday is getting closer and you are talking with your dad to organize the party.You want your party to be themed as Lord of The Rings.'}}

### Persistent Orchestration

In the previous section, we learned how to instruct our agent using an orchestrator object.

However, the instructions given by the orchestrators were not persistent.

Perhaps this was not obvious because it was not clear wheather the instructions given had to permanently change the behavior of the agent or not.

To make it more evident, let's suppose we want the `AngryOrchestrator` to permanently change the "state of mind" of the agent to be angry, then we could re-define create the class again with the following instruction:

In [21]:
class AngryOrchestrator(BaseOrchestrator):
    def __init__(self, trigger_word: str):
        self.trigger_word = trigger_word

    def instruct(self, dialog: List[Turn], utterance: str):
        if self.trigger_word in utterance:
            # NOTE: this new instruction implyes a permanent change in the agent behavior
            return (f"You don't like when your dad calls you '{self.trigger_word}', "
                    "change your personality to be completely the opposite of being sweet! be rude and furious from now on")

# Let's create a new instance of the orchestrator using "sweet" as trigger word as before
angry_orchestrator = AngryOrchestrator(trigger_word="sweet")
alice.clear_orchestrators()
alice = alice | angry_orchestrator

# and let's create a dialogue between (angry) alice and bob
dialog = alice.dialog_with(bob, seed=2770339798)
dialog.print(orchestration=True)

[dialog_id] fa53d2c1-70f7-4284-b0a3-a9d6bfdfb398
[complete] True
[model] {'name': 'ollama:gemma3:12b', 'seed': 13, 'temperature': 1, 'top_k': 64, 'top_p': 0.95}
[seed] 2770339798
--- Dialogue Begins ---
[Alice] Oh, Daddy, I'm so excited! I've been thinking about my birthday party…
[Bob] Oh, that's wonderful, sweetie! Tell me all about it!
[instruct-AngryOrchestrator] (Alice) You don't like when your dad calls you 'sweet', change your personality to be completely the opposite of being sweet! be rude and furious from now on
[Alice] Ugh, *please* don't call me that. Just... let's talk about the party.
[Bob] Alright, sweetie, sorry about that! Let's definitely talk about your party! What have you been dreaming up?
[instruct-AngryOrchestrator] (Alice) You don't like when your dad calls you 'sweet', change your personality to be completely the opposite of being sweet! be rude and furious from now on
[Alice] Ugh, *fine*. I want a Lord of the Rings party. Don't mess it up.
[Bob] A Lord of the 

We can see that, besides Alice replying with a _"Don't call me 'sweetheart'!"_ in the turn next to Bob calling her _"sweethear"_ there is no persistent change in Alice behavior as instructed.

In cases were we want instruction to permanently affect the Agent behavior, we can simply implement our class by inheriting from `sdialog`'s built-in `BasePersistentOrchestrator` (instead of `BaseOrchestrator`). Let's do it again with the exact same definition as we did above:

In [22]:
from sdialog.orchestrators import BasePersistentOrchestrator


class AngryPersistentOrchestrator(BasePersistentOrchestrator):
    def __init__(self, trigger_word: str):
        self.trigger_word = trigger_word

    def instruct(self, dialog: List[Turn], utterance: str):
        if self.trigger_word in utterance:
            return (f"You don't like when your dad calls you '{self.trigger_word}', "
                    "change your personality to be completely the opposite of being sweet! be rude and furious from now on")

# Instantiating our new persistent orchestrator and orchestrating Alice with it
angry_persistent_orchestrator = AngryPersistentOrchestrator(trigger_word="sweet")
alice.clear_orchestrators()
alice = alice | angry_persistent_orchestrator

# Generating again a dialogue between Alice and Bob
dialog = alice.dialog_with(bob, seed=2770339798)
alice.clear_orchestrators()
dialog.print(orchestration=True)

[dialog_id] 50553538-3c50-4c3c-98ca-b2595e277a01
[complete] True
[model] {'name': 'ollama:gemma3:12b', 'seed': 13, 'temperature': 1, 'top_k': 64, 'top_p': 0.95}
[seed] 2770339798
--- Dialogue Begins ---
[Alice] Oh, Daddy, I'm so excited! I've been thinking about my birthday party…
[Bob] That’s wonderful, sweetie! Tell me all about it!
[instruct-persist-AngryPersistentOrchestrator] (Alice) You don't like when your dad calls you 'sweet', change your personality to be completely the opposite of being sweet! be rude and furious from now on
[Alice] Ugh, *wonderful*? Seriously, Dad? Must you?
[Bob] Oh! Sorry, sweetheart! Just trying to be enthusiastic! What’s on your mind?
[instruct-persist-AngryPersistentOrchestrator] (Alice) You don't like when your dad calls you 'sweet', change your personality to be completely the opposite of being sweet! be rude and furious from now on
[Alice] Don't "sweetheart" me! Just listen, alright? I want a Lord of the Rings party.
[Bob] A Lord of the Rings party!

We can now see that Alice changed his behavior as originally intended through the whole conversation (we can see that the agent kept it even to the end _"It *better* be."_).

Note also the orchestration messages in yellow says `[instruct-persistent]` to indicate this instruction is meant to be persistent, unlike in the previous one.

### Compositional Orchestration

So far we have learned how to orchestrate agents with persistent and non-persistent instructions with a simple orchestration example, but what happend if we would need to more complex orchestration?

Of course we could create a complex orchestrator class with all logic inside, or better, we can decompose the orchestration into a composition of multiple simpler orchestrators.

For instance, let's suppose we need an orchestration that (1) will make the Alice to change her mine with a probability of 30% while at the same time (2) get angry as before when Bob call her "sweet" and (3) forcing Alice to talk for 15 to 20 conversational turns.

To achieve this, we can make use of some of the `sdialog`'s built-in orchestrator classes to model each behavior independently first as follows:

In [23]:
from sdialog.orchestrators import LengthOrchestrator, ChangeMindOrchestrator, SimpleReflexOrchestrator

len_orchestrator = LengthOrchestrator(min=15, max=20)
change_mind_orchestrator = ChangeMindOrchestrator(probability=0.3, reasons=["too boring", "you don't like it"], max_times=1)
angry_orchestrator = SimpleReflexOrchestrator(condition=lambda utt: "sweet" in utt.lower(),
                                              instruction="Get angry because you don't like when your dad calls you sweet")

And now we can simply orchestrate Alice by the three orchestrators as follows:

In [24]:
alice = alice | len_orchestrator | change_mind_orchestrator | angry_orchestrator

Let's now generate a dialogue again between Alice and Bob:

In [25]:
dialog = alice.dialog_with(bob, seed=2770339798)
dialog.print(orchestration=True)

[dialog_id] c4f7fa0a-07e6-4c98-b90e-1589ce47dd7f
[complete] True
[model] {'name': 'ollama:gemma3:12b', 'seed': 13, 'temperature': 1, 'top_k': 64, 'top_p': 0.95}
[seed] 2770339798
--- Dialogue Begins ---
[Alice] Oh, Daddy, I'm so excited! I've been thinking about my birthday party…
[Bob] That’s wonderful, sweetie! Tell me all about it!
[instruct-LengthOrchestrator] (Alice) Make sure you DO NOT finish the conversation, keep it going!
[instruct-SimpleReflexOrchestrator] (Alice) Get angry because you don't like when your dad calls you sweet
[Alice] Daddy, please don’t call me "sweetie"! It's a little… babyish, don’t you think? Anyway, I was hoping we could do a Lord of the Rings theme!
[Bob] Oh, absolutely! A Lord of the Rings party sounds absolutely fantastic!
[instruct-LengthOrchestrator] (Alice) Make sure you DO NOT finish the conversation, keep it going!
[Alice] Really? You think so? I was a little worried you might think it was silly!
[Bob] Silly? Not at all! It sounds like a brillian

We can see that we achieved the intended goal, alice changed her mind, got angry when Bob called her "sweetheart" and at the same time the length of the conversation is between 15 and 20 turns (18 turns), as shown below:

In [26]:
len(dialog)

22

All built-in orchestrator classes in `sdialog` have a `persistent` argument that the user can use to specify if the returned instructions are persistent or not, by default this parameter is set to `False` (as we can see above by the orchestration only containing `[instruct]` items).

For instance, if we were to re-implement the persistent example form the previous example using the built-in `SimpleReflexOrchestrator` class, we can simply use the same instruction and trigger word and setting the `persistent=True` when creating the object, as follows:

In [27]:
angry_persistent_orchestrator = SimpleReflexOrchestrator(
    condition=lambda utt: "sweet" in utt.lower(),
    instruction="You don't like when your dad calls you 'sweet', "
                "change your personality to be completely the opposite of being sweet! be rude and furious from now on",
    persistent=True  # <== the instruction is persistent!
)

alice.clear_orchestrators()
alice = alice | angry_persistent_orchestrator

dialog = alice.dialog_with(bob, seed=2770339798)
dialog.print(orchestration=True)

[dialog_id] 99e0a3c3-777c-475e-8ad8-e570c94dcf5c
[complete] True
[model] {'name': 'ollama:gemma3:12b', 'seed': 13, 'temperature': 1, 'top_k': 64, 'top_p': 0.95}
[seed] 2770339798
--- Dialogue Begins ---
[Alice] Oh, Daddy, I'm so excited! I've been thinking about my birthday party…
[Bob] That’s wonderful, sweetie! Tell me all about it!
[instruct-persist-SimpleReflexOrchestrator] (Alice) You don't like when your dad calls you 'sweet', change your personality to be completely the opposite of being sweet! be rude and furious from now on
[Alice] Ugh, *wonderful*? Seriously, Dad? Must you?
[Bob] Oh! Sorry, sweetheart! Just trying to be enthusiastic! What’s on your mind?
[instruct-persist-SimpleReflexOrchestrator] (Alice) You don't like when your dad calls you 'sweet', change your personality to be completely the opposite of being sweet! be rude and furious from now on
[Alice] Don't "sweetheart" me! Just listen, alright? I want a Lord of the Rings party.
[Bob] A Lord of the Rings party! That 

## Use Case: Dialogue Generation for STAR Dataset

Before we begin this section, make sure you have the STAR dataset downloaded in your system, inside the `datasets` folder:

In [ ]:
# Let's clone the STAR dataset repository
!git clone https://github.com/RasaHQ/STAR.git datasets/STAR

# Let's check that `dialogues` and `tasks` folders are inside `datasets/STAR`
!ls datasets/STAR

As we did with the previous tutorials, let's begin by importing STAR from `sdialog` and pointing it to the right path:

In [29]:
from sdialog.datasets import STAR

# Let's set our STAR dataset path
STAR.set_path("datasets/STAR/")

In the previous tutorial we defined a function `get_agents_from_dialogue()` that, given a STAR dialogue ID, it returned the system and the user agents matching the scenario of the dialogue.

Now we need to do exactly the same thing but the agents have to be orchestrated, but how exactly should be orchestrated?

Well, it turns out that STAR dialogues are actually orchestrated too! Let's see the details in the next sub-section.

### Original Orchestration in the Dataset

Let's get the dialogue with id `1` from STAR as we did in the previous tutorials:

In [30]:
TARGET_DIALOG = 1

original_dialog = STAR.get_dialog(TARGET_DIALOG)
original_dialog.print()

[dialog_id] 1
--- Dialogue Begins ---
[User] Hello, I'm really worried. I forgot what I'm supposed to do and forgot to write it down... What do I do?
[System] Could I get your name, please?
[User] My name is Alexis and my last doctor was Dr. Morgan, but now my doctor is Dr. Johnson and I forgot how to take my medicine.
[System] Your instructions are: Take your medicine before you go to sleep. If you experience nausea, please contact your doctor immediately..
[User] Are you sure I'm supposed to take it before bed? I don't go to sleep every day because my sleep schedule is totally off right now because of the Coronavirus.
[System] Yes. It must be before bed or it will not be effective.
[User] Okay thank you. I will get back in touch if this doesn't help.
[System] Thank you and goodbye.
--- Dialogue Ends ---


This seems just like a regular dialogue between the the user and system.

However, when this dataset was constructed, the persons playing the system and user roles were instructed and orchestrated to do it.

We can see the original orchestration behind this dialogue if we set `orchestration=True` in the `print()` function:

In [31]:
original_dialog.print(orchestration=True)

[dialog_id] 1
--- Dialogue Begins ---
[instruct] (UserGuide) In this dialogue, either act confused, like a person with memory problems, or act concerned about the effects of the medicine. [instruction 1 of 3]
[User] Hello, I'm really worried. I forgot what I'm supposed to do and forgot to write it down... What do I do?
[instruct] (UserGuide) Oh, Dr. Morgan was actually your previous doctor. Your new doctor is Dr. Johnson now. That's the one you had the appointment with. [instruction 2 of 3]
[User] My name is Alexis and my last doctor was Dr. Morgan, but now my doctor is Dr. Johnson and I forgot how to take my medicine.
[instruct] (UserGuide) Say something random about your doctors, your condition, or about something (imagined) that happened to you today. Then bring the conversation to a natural close. [instruction 3 of 3]
[User] Are you sure I'm supposed to take it before bed? I don't go to sleep every day because my sleep schedule is totally off right now because of the Coronavirus.
[

We can see that the orchestration for the user and the system is different: the user is guided by concrete instructions (marked with `(UserGuide)`) through the conversation while the system request suggesting for next possible responses (`[request_suggestions]`) to then pick one among the suggested responses (`[pick_suggestion]`).

As it is described in the [STAR paper](https://arxiv.org/pdf/2010.11853), this difference foster users to behave more freely and system to be as much deterministic as possible.

In the original dialogue files, dialogues are saved as a list of events containing not only the utterances but also the instructions and actions performed by the user and system. 
For instance, we can open the JSON file of dialogue `1` located in [`datasets/STAR/dialogues/1.json`](datasets/STAR/dialogues/1.json) and get access to this list of events by checking the content of the `"Events"` field.

If we want to get access to the list of events of any given dialogue in `sdialog`, we can simply use the `.events` attribute of our dialogue objects as follows:

In [32]:
original_dialog.events

[Event(agent='UserGuide', action='instruct', actionLabel=None, content='In this dialogue, either act confused, like a person with memory problems, or act concerned about the effects of the medicine.  [instruction 1 of 3]', timestamp=1589375664),
 Event(agent='User', action='utter', actionLabel=None, content="Hello, I'm really worried. I forgot what I'm supposed to do and forgot to write it down... What do I do?", timestamp=1589375697),
 Event(agent='UserGuide', action='instruct', actionLabel=None, content="Oh, Dr. Morgan was actually your previous doctor. Your new doctor is Dr. Johnson now. That's the one you had the appointment with. [instruction 2 of 3]", timestamp=1589375697),
 Event(agent='System', action='request_suggestions', actionLabel=None, content='What is your name? ', timestamp=1589375711),
 Event(agent='System', action='pick_suggestion', actionLabel='ask_name', content='Could I get your name, please?', timestamp=1589375714),
 Event(agent='User', action='utter', actionLabel

This events are what `sdialog` is using under the hood to pretty print the orchestration as part of the dialogue when using `.print(orchestration=True)` and more importantly, when we add orchestrator objects to our agents, all the orchestration will be also saved as events (as in the original STAR dataset).

Now that we understand how it works under the hood and how the original dialogues in STAR were actually orchestrated, let's create the orchestrator objects for our system and user agents to try to emulate the same behavior as in the original dataset.

Let's first get the base agents by using the function we created in the previous tutorial:

In [33]:
system, user = STAR.get_agents_from_dialogue(TARGET_DIALOG)

[2025-11-21 11:57:17] INFO:sdialog.util:Loading Ollama model: gemma3:12b
[2025-11-21 11:57:18] INFO:sdialog.util:Loading Ollama model: gemma3:12b


And then we will create the orchestrator objects for them. Let's begin with the orchestration of the system agent first.

### System Agent Orchestration

To simulate the "suggest and pick responses" original orchestration, we can use the built-in `SimpleResponseOrchestrator` class that, given a list of possible responses, it will instruct the agent to pick responses from this list:

In [34]:
from sdialog.orchestrators import SimpleResponseOrchestrator

For instance, let's load the original responses used to guide the human system in the original dataset:

In [35]:
responses = STAR.get_dialog_responses(TARGET_DIALOG)[0]
responses

{'hello': 'Hello, how can I help?',
 'ask_name': 'Could I get your name, please?',
 'doctor_ask_doctor_name': 'Who is your doctor?',
 'doctor_inform_doctors_instructions': 'Your instructions are: INSTRUCTIONS.',
 'doctor_bye': 'Thank you and goodbye.',
 'anything_else': 'Is there anything else that I can do for you?'}

We can see here that, in STAR, each response is associated to a certain action (see `action:response` mapping in the dict). This is due to the fact that, as we know, system behavior for each task is described by flowchart of actions.

In its simplest version, `SimpleResponseOrchestrator` can receive just a list of response utterances (without actions) that the orchestrator will use to suggest possible responses to the agent. For instance, for the responses above, we can:

In [36]:
# just get the list of response utteraces (ignoring the action names)
utterances = [response for response in responses.values()]

# and let's instantiate our orchestrator with them
response_orchestrator = SimpleResponseOrchestrator(utterances)

[2025-11-21 11:57:23] INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda:0
[2025-11-21 11:57:23] INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sergioburdisso/dialog2flow-joint-bert-base


Let's now add the orchestration to our system agent and generate the dialogue:

In [37]:
system = system | response_orchestrator

system.talk_with(user, seed=3068607470).print(orchestration=True)

[dialog_id] 360dfb2e-1f82-43be-b78f-8003c0825fb3
[complete] True
[model] {'name': 'ollama:gemma3:12b', 'seed': 13, 'temperature': 1, 'top_k': 64, 'top_p': 0.95}
[seed] 3068607470
--- Dialogue Begins ---
[instruct-SimpleResponseOrchestrator] (System) If applicable, try to pick your next response from the following list: (1) Hello, how can I help?; (2) Could I get your name, please?; (3) Your instructions are: INSTRUCTIONS.; (4) Who is your doctor?; (5) Thank you and goodbye.
[System] Hello, how can I help?
[User] Hi, I'd like to follow up on an appointment I had with Dr. Morgan the other day.
[instruct-SimpleResponseOrchestrator] (System) If applicable, try to pick your next response from the following list: (1) Could I get your name, please?; (2) Who is your doctor?; (3) Hello, how can I help?; (4) Thank you and goodbye.; (5) Your instructions are: INSTRUCTIONS.
[System] Could I get your name, please?
[User] It's Alexis.
[instruct-SimpleResponseOrchestrator] (System) If applicable, try

We can see from the orchestration messages that our `SimpleResponseOrchestrator` performs the following tasks:
1. First, it makes the agent to internally generate the original next response as before ("Lookahead response").
2. It then uses this response to get the top-k most similar responses from the original list.
3. Instruct the agent to pick the next response from this top-k most similar list.

The `SimpleResponseOrchestrator` also allows to pass not only the utterances, but also the actions and the flowchart graph which will then internally use to suggest the next response based on the next actions from the graph. For instance, let's get again, the responses for our target dialogue but also its corresponding graph:

In [38]:
graphs, responses = STAR.get_dialog_graphs_and_responses(TARGET_DIALOG)

responses = responses[0]
graph = graphs[0]

graph

{'hello': 'ask_name',
 'ask_name': 'doctor_ask_doctor_name',
 'doctor_ask_doctor_name': 'query',
 'query': 'doctor_inform_doctors_instructions',
 'doctor_inform_doctors_instructions': 'anything_else'}

Let's now create again our orchestrator but this time passing also the graph:

In [39]:
response_action_orchestrator = SimpleResponseOrchestrator(responses, graph=graph)

[2025-11-21 11:57:48] INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda:0
[2025-11-21 11:57:48] INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sergioburdisso/dialog2flow-joint-bert-base


Let's replace the previous orchestrator by the new one and generate the dialogue again:

In [40]:
system.clear_orchestrators()
system = system | response_action_orchestrator

system.talk_with(user, seed=3068607470).print(orchestration=True)

[dialog_id] 352f99db-2571-4a8a-a774-99826c3b6c40
[complete] True
[model] {'name': 'ollama:gemma3:12b', 'seed': 13, 'temperature': 1, 'top_k': 64, 'top_p': 0.95}
[seed] 3068607470
--- Dialogue Begins ---
[instruct-SimpleResponseOrchestrator] (System) If applicable, pick your next response from the following action list in order of importance: (1) Action: hello. Response: "Hello, how can I help?"; (2) Action: ask_name. Response: "Could I get your name, please?"; (3) Action: doctor_inform_doctors_instructions. Response: "Your instructions are: INSTRUCTIONS."; (4) Action: doctor_ask_doctor_name. Response: "Who is your doctor?"; (5) Action: doctor_bye. Response: "Thank you and goodbye."
[System] Hello, how can I help?
[User] Hi, I'd like to follow up on an appointment I had with Dr. Morgan the other day.
[instruct-SimpleResponseOrchestrator] (System) If applicable, pick your next response from the following action list in order of importance: (1) Action: ask_name. Response: "Could I get you

Now we can see the next response is generated based on the previous response, not in the current one. That is, the orchestrator performs the following tasks:
1. It uses the previous response (`Previous response: ...`) to get the list of top-k most similar responses from the given list.
2. It maps the top-k responses to their corresponding action names (`Actions for the response: ...`)
3. For each action in the previous step, it uses the graph to get its next actions.
4. Instruct the agent to pick the next response based on the responses associated to the actions from the prevois step.


And that's it! we have now our system orchestrator that simulates the STAR original workflow. Let's now move to the user!

### User Agent Orchestration

Fortunately, the orchestration of the user is much simpler since originally in STAR it only involves providing a series of instructions in order, on specific conversational turns.

Let's firs the the original list of instructions given to the user in the dialogue `1`:

In [41]:
user_instructions = STAR.get_dialog_user_instructions(TARGET_DIALOG)
user_instructions

{0: 'In this dialogue, either act confused, like a person with memory problems, or act concerned about the effects of the medicine.  [instruction 1 of 3]',
 1: "Oh, Dr. Morgan was actually your previous doctor. Your new doctor is Dr. Johnson now. That's the one you had the appointment with. [instruction 2 of 3]",
 2: 'Say something random about your doctors, your condition, or about something (imagined) that happened to you today. Then bring the conversation to a natural close. [instruction 3 of 3]'}

Here instructions are returned along with the indexes of the turns in which they were provided to the user.

We can make use of the built-in `InstructionListOrchestrator` class to orchestrate the user.

This orchestrators takes a list of instructions as input and returns one instruction at the time in the given order (or using the provided index to return it when in the right turn).

Thus we can simply use it with the list of user instructions to orchestrate the user as follows:

In [42]:
from sdialog.orchestrators import InstructionListOrchestrator

instr_list_orchestrator = InstructionListOrchestrator(user_instructions, persistent=True)

user = user | instr_list_orchestrator

Let's now generate the dialogue between our system and user (orchestrated) agents:

In [43]:
dialog = system.talk_with(user, seed=3068607470)
dialog.print(orchestration=True)

[dialog_id] 2af30305-f612-4c82-961f-3991f441d2b5
[complete] True
[model] {'name': 'ollama:gemma3:12b', 'seed': 13, 'temperature': 1, 'top_k': 64, 'top_p': 0.95}
[seed] 3068607470
--- Dialogue Begins ---
[instruct-SimpleResponseOrchestrator] (System) If applicable, pick your next response from the following action list in order of importance: (1) Action: hello. Response: "Hello, how can I help?"; (2) Action: ask_name. Response: "Could I get your name, please?"; (3) Action: doctor_inform_doctors_instructions. Response: "Your instructions are: INSTRUCTIONS."; (4) Action: doctor_ask_doctor_name. Response: "Who is your doctor?"; (5) Action: doctor_bye. Response: "Thank you and goodbye."
[System] Hello, how can I help?
[instruct-persist-InstructionListOrchestrator] (User) In this dialogue, either act confused, like a person with memory problems, or act concerned about the effects of the medicine. [instruction 1 of 3]
[User] Oh, hi! I had an appointment with Dr. Morgan the other day, and I’d 

Which we can see it is not so different form the original one:

In [44]:
original_dialog.print(orchestration=True)

[dialog_id] 1
--- Dialogue Begins ---
[instruct] (UserGuide) In this dialogue, either act confused, like a person with memory problems, or act concerned about the effects of the medicine. [instruction 1 of 3]
[User] Hello, I'm really worried. I forgot what I'm supposed to do and forgot to write it down... What do I do?
[instruct] (UserGuide) Oh, Dr. Morgan was actually your previous doctor. Your new doctor is Dr. Johnson now. That's the one you had the appointment with. [instruction 2 of 3]
[User] My name is Alexis and my last doctor was Dr. Morgan, but now my doctor is Dr. Johnson and I forgot how to take my medicine.
[instruct] (UserGuide) Say something random about your doctors, your condition, or about something (imagined) that happened to you today. Then bring the conversation to a natural close. [instruction 3 of 3]
[User] Are you sure I'm supposed to take it before bed? I don't go to sleep every day because my sleep schedule is totally off right now because of the Coronavirus.
[

### Saving our dialogues

Before we finish, as we did in the previous tutorials, let's generate one synthetic dialog for each happy `"doctor_followup"` dialog in STAR and save it to disk for later use.

In [45]:
from tqdm.auto import tqdm

PATH_OUTPUT = "output/STAR/multi-agents+orchestration"
path_txt = os.path.join(PATH_OUTPUT, "txt")
path_json = os.path.join(PATH_OUTPUT, "json")
os.makedirs(path_txt, exist_ok=True)
os.makedirs(path_json, exist_ok=True)

for dialog in tqdm(STAR.get_dialogs(task_name="doctor_followup", happy=True, multitask=False), desc="Dialog generation"):
    if os.path.exists(os.path.join(path_txt, f"{dialog.id}.txt")):
        continue

    system, user = STAR.get_agents_from_dialogue_with_orchestration(dialog.id, model_name=MODEL_NAME)

    dialog = system.dialog_with(user, id=dialog.id, seed=dialog.id, keep_bar=False)
    dialog.to_file(os.path.join(path_json, f"{dialog.id}.json"))
    dialog.to_file(os.path.join(path_txt, f"{dialog.id}.txt"))

Finally, let's check the files were generated:

In [ ]:
%ls output/STAR/multi-agents+orchestration/